In [1]:
import random
import numpy as np
import torch
import torch.nn as nn
from torchsummary import summary
import torchvision.models as models

# Assign seed to ensure repeatability
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# Define Residual Blocks

There are two types of residual blocks:
- Basic Block. Used for ResNet18 and ResNet34
- Bottleneck Block. Used for ResNet50, ResNet101, ResNet152

In [2]:
class BasicBlock(nn.Module):
  expansion = 1
  def __init__(self, in_channels, out_channels, stride=1, downsample=None):
    super().__init__()
    # Declare the elements of a basic block
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                           stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.act = nn.ReLU(inplace=True)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                           stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.downsample = downsample

  def forward(self, x):

    identity_map = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.act(out)
    out = self.conv2(out)
    out = self.bn2(out)
    if self.downsample is not None:
      identity_map = self.downsample(x)
    # Add the identity map to the output of the convolution
    out += identity_map
    out = self.act(out)

    return out

In [3]:
basic_block = BasicBlock(3, 64)
basic_block

BasicBlock(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [4]:
class BottleneckBlock(nn.Module):
  expansion = 4
  def __init__(self, in_channels, out_channels, stride=1, downsample=None):
    super().__init__()
    # Declare the elements of a basic block
    self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1,
                           stride=stride, bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.act = nn.ReLU(inplace=True)
    self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                           stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, 
                           kernel_size=1, stride=1, bias=False)
    self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
    self.downsample = downsample

  def forward(self, x):

    identity_map = x

    out = self.conv1(x)
    out = self.bn1(out)
    out = self.act(out)
    out = self.conv2(out)
    out = self.bn2(out)
    out = self.act(out)
    out = self.conv3(out)
    out = self.bn3(out)
    if self.downsample is not None:
      identity_map = self.downsample(x)
    # Add the identity map to the output of the convolution
    out += identity_map
    out = self.act(out)

    return out

In [5]:
bottleneck_block = BottleneckBlock(64, 64)
bottleneck_block

BottleneckBlock(
  (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act): ReLU(inplace=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

# Define ResNet Layers

In [6]:
class ResNetLayer(nn.Module):
  def __init__(self, block, in_channels, out_channels, n_blocks, stride=1):
    super().__init__()
    downsample = None
    if stride != 1 or in_channels != out_channels * block.expansion:
      downsample = nn.Sequential(nn.Conv2d(in_channels, 
                                           out_channels * block.expansion, 
                                           kernel_size=1, stride=stride,
                                           bias=False),
                                 nn.BatchNorm2d(out_channels * block.expansion))
    layers = []
    layers.append(block(in_channels, out_channels, stride, downsample))
    in_channels = out_channels * block.expansion
    for _ in range(1, n_blocks):
      layers.append(block(in_channels, out_channels))
    self.blocks = nn.Sequential(*layers)

  def forward(self, x):

    x = self.blocks(x)
    
    return x

In [7]:
resnet_layer1 = ResNetLayer(BasicBlock, 64, 64, 3)
resnet_layer1

ResNetLayer(
  (blocks): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1),

In [8]:
resnet_layer2 = ResNetLayer(BottleneckBlock, 64, 128, 4, stride=2)
resnet_layer2

ResNetLayer(
  (blocks): Sequential(
    (0): BottleneckBlock(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU(inplace=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(64, 512, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BottleneckBlock(
      (conv1): Conv2d(512, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=

# Define ResNet model

We have defined the key elements of the ResNet model. Now, we will piece together the different elements to define the ResNet model architecture.

In [9]:
class ResNet(nn.Module):
  def __init__(self, block, layers, n_classes):
    super().__init__()
    self.n_filters = 64
    # Define the convolution block
    self.conv1 = nn.Conv2d(3, self.n_filters, kernel_size=7, stride=2, padding=3, 
                           bias=False)
    self.bn1 = nn.BatchNorm2d(self.n_filters)
    self.relu = nn.ReLU(inplace=True)
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    # Define the layers
    self.layer1 = ResNetLayer(block, self.n_filters, 64, layers[0])
    self.layer2 = ResNetLayer(block, 64 * block.expansion, 128, 
                              layers[1], stride=2)
    self.layer3 = ResNetLayer(block, 128 * block.expansion, 256, 
                              layers[2], stride=2)
    self.layer4 = ResNetLayer(block, 256 * block.expansion, 512, 
                              layers[3], stride=2)

    # Average of activations across a convolutional grid
    self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
    # Final fully connected layer
    self.fc = nn.Linear(512 * block.expansion, n_classes)

  def forward(self, x):

    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avg_pool(x)
    x = torch.flatten(x, 1)
    x = self.fc(x)

    return x

In [10]:
def resnet(name='resnet50', n_classes=1000):

  if name == 'resnet18':
    layers = [2, 2, 2, 2]
    block = BasicBlock
  elif name == 'resnet34':
    layers = [3, 4, 6, 3]
    block = BasicBlock
  elif name == 'resnet50':
    layers = [3, 4, 6, 3]
    block = BottleneckBlock
  elif name == 'resnet101':
    layers = [3, 4, 23, 3]
    block = BottleneckBlock
  elif name == 'resnet152':
    layers = [3, 8, 36, 3]
    block = BottleneckBlock

  # Initialize seed for repeatability
  torch.manual_seed(SEED)

  # Call the model
  model = ResNet(block, layers, n_classes)
  
  return model

# Check if our implementation matches the ResNet models available via Torch Vision

## ResNet-18

In [11]:
resnet18_model = resnet('resnet18', n_classes=1000)
print("Number of parameters in our ResNet-18 model:", sum(np.prod(parameters.shape)
  for parameters in resnet18_model.parameters()))
print("Number of parameters in PyTorch ResNet-18 model:", sum(np.prod(parameters.shape)
  for parameters in models.resnet18(False).parameters()))

Number of parameters in our ResNet-18 model: 11689512
Number of parameters in PyTorch ResNet-18 model: 11689512


## ResNet-34

In [12]:
resnet34_model = resnet('resnet34', n_classes=1000)
print("Number of parameters in our ResNet-34 model:", sum(np.prod(parameters.shape)
  for parameters in resnet34_model.parameters()))
print("Number of parameters in PyTorch ResNet-34 model:", sum(np.prod(parameters.shape)
  for parameters in models.resnet34(False).parameters()))

Number of parameters in our ResNet-34 model: 21797672
Number of parameters in PyTorch ResNet-34 model: 21797672


## ResNet-50

In [13]:
resnet50_model = resnet('resnet50', n_classes=1000)
print("Number of parameters in our ResNet-50 model:", sum(np.prod(parameters.shape)
  for parameters in resnet50_model.parameters()))
print("Number of parameters in PyTorch ResNet-50 model:", sum(np.prod(parameters.shape)
  for parameters in models.resnet50(False).parameters()))

Number of parameters in our ResNet-50 model: 25557032
Number of parameters in PyTorch ResNet-50 model: 25557032


## ResNet-101

In [14]:
resnet101_model = resnet('resnet101', n_classes=1000)
print("Number of parameters in our ResNet-101 model:", sum(np.prod(parameters.shape)
  for parameters in resnet101_model.parameters()))
print("Number of parameters in PyTorch ResNet-101 model:", sum(np.prod(parameters.shape)
  for parameters in models.resnet101(False).parameters()))

Number of parameters in our ResNet-101 model: 44549160
Number of parameters in PyTorch ResNet-101 model: 44549160


## ResNet-152

In [15]:
resnet152_model = resnet('resnet152', n_classes=1000)
print("Number of parameters in our ResNet-152 model:", sum(np.prod(parameters.shape)
  for parameters in resnet152_model.parameters()))
print("Number of parameters in PyTorch ResNet-152 model:", sum(np.prod(parameters.shape)
  for parameters in models.resnet152(False).parameters()))

Number of parameters in our ResNet-152 model: 60192808
Number of parameters in PyTorch ResNet-152 model: 60192808


# References

[1] [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)

[2] [Torch Vision - ResNet](https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py)

[3] [Convolutional Neural Networks’ mathematics](https://towardsdatascience.com/convolutional-neural-networks-mathematics-1beb3e6447c0)

[4] [ResNet, torchvision, bottlenecks, and layers not as they seem.](https://erikgaas.medium.com/resnet-torchvision-bottlenecks-and-layers-not-as-they-seem-145620f93096)

[5] [Residual Networks: Implementing ResNet in Pytorch](https://towardsdatascience.com/residual-network-implementing-resnet-a7da63c7b278)

[6] [Understanding and Building Resnet from scratch using Pytorch.](https://jarvislabs.ai/blogs/resnet)

[7] [Building ResNet34 in PyTorch](https://github.com/jarvislabsai/blog/blob/master/build_resnet34_pytorch/Building%20Resnet%20in%20PyTorch.ipynb)